<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/neo4jvector_langchain_deepdive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai wikipedia tiktoken neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 20.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=f4eebad20b3e578d907d048fa9e0c110d9a28724fee3f28b8cd35ef7e6246838
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
  Created wheel for neo4j: filename=neo4j-5.11.0-py3-none-any.whl size=259793 sha256=e465b3e7b9a0f93d1b5d14621d68340e64

In [2]:
import os

from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

os.environ["OPENAI_API_KEY"] = "sk-"

In [3]:
# Read the wikipedia article
raw_documents = WikipediaLoader(query="The Witcher").load()
# Define chunking strategy
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=20
)
# Chunk the document
documents = text_splitter.split_documents(raw_documents)
for d in documents:
    del d.metadata["summary"]

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [5]:
url = "neo4j+s://.databases.neo4j.io"
username = "neo4j"
password = "-23VkKvkuXDGne9M"

neo4j_db = Neo4jVector.from_documents(
    documents,
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    database="neo4j",  # neo4j by default
    index_name="wikipedia",  # vector by default
    node_label="WikipediaArticle",  # Chunk by default
    text_node_property="info",  # text by default
    embedding_node_property="vector",  # embedding by default
    create_id_index=True,  # True by default
)

In [6]:
neo4j_db.query("SHOW CONSTRAINTS")

[{'id': 4,
  'name': 'constraint_e5da4d45',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['WikipediaArticle'],
  'properties': ['id'],
  'ownedIndex': 'constraint_e5da4d45',
  'propertyType': None}]

In [7]:
neo4j_db.query(
    """SHOW INDEXES
       YIELD name, type, labelsOrTypes, properties, options
       WHERE type = 'VECTOR'
    """
)

[{'name': 'wikipedia',
  'type': 'VECTOR',
  'labelsOrTypes': ['WikipediaArticle'],
  'properties': ['vector'],
  'options': {'indexProvider': 'vector-1.0',
   'indexConfig': {'vector.dimensions': 1536,
    'vector.similarity_function': 'cosine'}}}]

In [8]:
query = "Who wrote the Witcher?"

results = neo4j_db.similarity_search(query, k=1)

# Print only the first paragraph for blog post purposes
print(results[0].page_content.split("\n\n")[0])

The Witcher (Polish: Wiedźmin pronounced [ˈvʲɛd͡ʑmʲin]) is a series of six fantasy novels and 15 short stories written by Polish author Andrzej Sapkowski. The series revolves around the eponymous "witcher", Geralt of Rivia. In Sapkowski's works, "witchers" are beast hunters who are given supernatural abilities at a young age to battle wild beasts and monsters. The Witcher began with a titular 1986 short story that Sapkowski entered into a competition held by Fantastyka magazine, marking his debut as an author. Due to reader demand, Sapkowski wrote 14 more stories before starting a series of novels in 1994. Known as The Witcher Saga, he wrote one book a year until the fifth and final installment in 1999. A standalone prequel novel, Season of Storms, was published in 2013.
The books have been described as having a cult following in Poland and Central and Eastern European countries. They have been translated into 37 languages and sold over 15 million copies worldwide as of December 2019. 

In [9]:
# Read the wikipedia article
raw_documents = WikipediaLoader(query="Arcane (TV series)").load()
# Define chunking strategy
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=20
)
# Chunk the document
arcane_documents = text_splitter.split_documents(raw_documents)
for d in arcane_documents:
    del d.metadata["summary"]

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [10]:
neo4j_db.add_documents(arcane_documents, ids=[x for x in range(len(arcane_documents))])

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [11]:
query = "What is the Arcane series about?"

results = neo4j_db.similarity_search(query, k=1)

# Print only the first paragraph for blog post purposes
print(results[0].page_content.split("\n\n")[0])

Arcane (titled onscreen as Arcane: League of Legends) is an adult animated steampunk action-adventure television series created by Christian Linke and Alex Yee. It was produced by the French animation studio Fortiche under the supervision of Riot Games, and distributed by Netflix. Set in Riot's League of Legends fictional universe, it primarily focuses on sisters Vi and Jinx. The series was announced at the League of Legends 10th anniversary celebration in 2019, and first released in November 2021. Following the conclusion of the first season, Riot Games and Netflix announced that a second season was in production.
Arcane's first season was released to universal acclaim, with praise directed at its animation, story, worldbuilding, action sequences, characters, emotional weight, music, and voice acting. Some have noted the series' appeal both to casual viewers who have never played League of Legends and to long-time fans of the game. It also set the record as Netflix's highest-rated ser

In [12]:
neo4j_db.query("MATCH (n:WikipediaArticle {id:5}) RETURN n {.*, vector:null}")

[{'n': {'vector': None,
   'id': 5,
   'source': 'https://en.wikipedia.org/wiki/Heimerdinger',
   'title': 'Heimerdinger',
   'info': 'Heimerdinger is a surname. Notable people with the surname include:\n\nChris Heimerdinger (born 1963), American author\nFriedrich Heimerdinger (1817–1882), German painter\nMike Heimerdinger (1952–2011), American football coach\nCecil B. Heimerdinger, a fictional character in League of Legends and Arcane (TV series)'}}]

In [13]:
existing_index = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name="wikipedia",
    text_node_property="info",  # Need to define if it is not default
)

In [14]:
print(existing_index.node_label)
print(existing_index.embedding_node_property)

WikipediaArticle
vector


In [15]:
existing_index.add_documents(
    [
        Document(
            page_content="LangChain is the coolest library since the Library of Alexandria"
        )
    ],
    ids=["langchain"],
)

['langchain']

In [16]:
existing_index.similarity_search("What do you know about LangChain?", k=1)

[Document(page_content='LangChain is the coolest library since the Library of Alexandria', metadata={})]

In [17]:
existing_index.query(
    """MATCH (w:WikipediaArticle {id:'langchain'})
       MERGE (w)<-[:EDITED_BY]-(:Person {name:"Tomaz"})
    """
)

[]

In [18]:
"""
read_query = (
    "CALL db.index.vector.queryNodes($index, $k, $embedding) "
    "YIELD node, score "
) + retrieval_query
"""

existing_index_return = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    database="neo4j",
    index_name="wikipedia",
    text_node_property="info",
    retrieval_query="""
       OPTIONAL MATCH (node)<-[:EDITED_BY]-(p)
       RETURN node.info AS text, score, {editors: collect(p.name)} AS metadata
    """,
)

In [19]:
existing_index_return.similarity_search("What do you know about LangChain?", k=1)

[Document(page_content='LangChain is the coolest library since the Library of Alexandria', metadata={'editors': ['Tomaz']})]